In [2]:
from py2neo import Graph
from pandas import DataFrame
from pprint import pprint

# Test

In [3]:
graph = Graph(host="localhost://7474", auth=("neo4j", "123456"))

### Find top3 and bottom 3 reviews contain the given food nodes list from one restaurant

In [28]:
#Query top 3 and Bottom 3 reviews for food
query = """
            MATCH (r:Review)-[a:About]->(re:Restaurant)
            WHERE re.name = "Tamoya Udon (Liang Court)"
            WITH re, r, ["beef","noodle"] as foods
            MATCH (r:Review)-[c:Contains]->(f:Food)
            WHERE f.name IN foods
            WITH re, r, count(distinct c) as co, foods
            WHERE co = size(foods)
            MATCH (r)-[a:About]->(re:Restaurant)
            WITH re, r,foods
            MATCH (r:Review)-[c:Contains]->(f:Food {name:foods[0]})
            RETURN re.name,r.review,(c.food_sentiment) AS senti
            ORDER BY senti DESC
            """

c=graph.run(query).data()
print('top 3 reviews:',c[:3],'\n')
print('bottom 3 reviews:',c[-3:])


top 3 reviews: [{'re.name': 'Tamoya Udon (Liang Court)', 'r.review': "Whenever I crave a good bowl of Udon, I almost always end up at @tamoyasg !:This is their Beef Udon ($10.80) that comes with a super generous portion of flavourful beef, albeit being on the fattier side. I also spammed a whole lot of wakame, spring onion and crispy tempura bits (yumz). If you're a big eater, you can also choose to upsize your bowl from a Regular to a Large at no extra cost! 😝 Their Udon is seriously one of the best I've had; thick with a nice springy bite. I don't usually drink much of the broth, but their broth is also super addictive that I finished almost every single drop of it! Their prices are already very affordable but with @entertainersg 's 1for1 , my meal was made even more pocket-friendly! 💯 #burpple #burpplesg #udon #japanesefood #japanesecuisine #noodles #entertainerapp #udonnoodles #beef #beefudon #tamoyaudon #whati8todaysg #whati8today #sgeats #eatmoresg #singaporefoodlisting #sgfoodbl

# Search query functions

In [98]:
def search_restaurant_by_cusine(graph, text, sort):
    result = []

    query_rating = """
        MATCH (r:Review)-[a:About]->(re:Restaurant)
        WHERE re.cuisine = {term}
        WITH re,COUNT(r) AS number,SUM(a.restaurant_sentiment) AS senti      

        RETURN re.name, senti/number AS average_senti
        ORDER BY average_senti DESC

    """

    query_popularity = """
        MATCH (r:Review)-[a:About]->(re:Restaurant)
        WHERE re.cuisine = {term}
        WITH re,COUNT(r) AS number        
        RETURN re.name,number
        ORDER BY number DESC

    """
    if sort == 'number_of_reviews':
        query = query_popularity

    if sort == 'rating':
        query = query_rating

    ans = graph.run(query, term=text).data()

    for item in ans:
        result.append(item['re.name'])

    if not result:
        return ['No records found']
    else:
        return result


def query_reviews_cuisine(graph, res_list):
    result = []
    query_top = """
                MATCH (r:Review)-[a:About]->(re:Restaurant)
                WHERE re.name = {term}
                WITH re, a, r
                ORDER BY a.restaurant_sentiment DESC
                WITH re, COLLECT(r)[0..3] AS top3
                UNWIND top3 AS top
                RETURN re.name, top.review
                """

    query_bottom = """
                MATCH (r:Review)-[a:About]->(re:Restaurant)
                WHERE re.name = {term}
                WITH re, a, r
                ORDER BY a.restaurant_sentiment DESC
                WITH re, COLLECT(r)[-4..-1] AS bottom3
                UNWIND bottom3 AS bottom
                RETURN re.name, bottom.review
                """
    for item in res_list:

        temp = [item]
        tmp_t = graph.run(query_top, term=item).data()
        for r in tmp_t:
            if not r['top.review']:
                temp.append('No reviews')
            else:
                temp.append(r['top.review'])

        tmp_b = graph.run(query_bottom, term=item).data()
        for r in tmp_b:
            if not r['bottom.review']:
                temp.append('No reviews')
            else:
                temp.append(r['bottom.review'])

        result.append(temp)

    return result


def query_reviews_food(graph, res_list, food, spli):
    result = []

    if not spli:
        query_top = """
                MATCH (r:Review)-[a:About]->(re:Restaurant)
                WHERE re.name = {restaurant}
                WITH re, r
                MATCH (r:Review)-[c:Contains]->(f:Food)
                WHERE f.name = {food}
                WITH re, r, c
                ORDER BY c.food_sentiment DESC
                WITH re, COLLECT(r)[0..3] AS top3
                UNWIND top3 AS top
                RETURN re.name, top.review
                """

        query_bottom = """
                MATCH (r:Review)-[a:About]->(re:Restaurant)
                WHERE re.name = {restaurant}
                WITH re, r
                MATCH (r:Review)-[c:Contains]->(f:Food)
                WHERE f.name = {food}
                WITH re, r, c
                ORDER BY c.food_sentiment DESC
                WITH re, COLLECT(r)[-4..-1] AS bottom3
                UNWIND bottom3 AS bottom
                RETURN re.name, bottom.review
                """
        for item in res_list:

            temp = [item]
            tmp_t = graph.run(query_top, restaurant=item, food=food).data()
            for r in tmp_t:
                if not r['top.review']:
                    temp.append('No reviews')
                else:
                    temp.append(r['top.review'])

            tmp_b = graph.run(query_bottom, restaurant=item, food=food).data()
            for r in tmp_b:
                if not r['bottom.review']:
                    temp.append('No reviews')
                else:
                    temp.append(r['bottom.review'])

            result.append(temp)

        return result

    else:
        food_split = food.split()

        query_2 = """
                MATCH (r:Review)-[a:About]->(re:Restaurant)
                WHERE re.name = {restaurant}
                WITH re, r, {food_list} as foods
                MATCH (r:Review)-[c:Contains]->(f:Food)
                WHERE f.name IN foods
                WITH re, r, count(distinct c) as co, foods
                WHERE co = size(foods)
                MATCH (r)-[a:About]->(re:Restaurant)
                WITH re, r,foods
                MATCH (r:Review)-[c:Contains]->(f:Food {name:foods[0]})
                RETURN re.name,r.review,(c.food_sentiment) AS senti
                ORDER BY senti DESC
                """

        for item in res_list:
            t = [item]
            tmp_r = graph.run(query_2, restaurant=item, food_list=food_split).data()
            temp = []
            for r in tmp_r:
                if not r['r.review']:
                    temp.append('No reviews')
                else:
                    temp.append(r['r.review'])
            t = t + temp[:3] + temp[-3:]
            result.append(t)

        return result


def search_restaurant_by_food(graph, text, sort):
    result = []
    key_words = text
    spli = False

    query_pop = """
        MATCH (r:Review)-[c:Contains]->(f:Food)
        WHERE f.name = {term}
        WITH DISTINCT r, c
        MATCH (r)-[a:About]->(re:Restaurant)
        RETURN re.name, sum(c.food_sentiment)/count(r) AS average_senti, count(r) AS number
        ORDER BY number DESC
        """

    query_rate = """
        MATCH (r:Review)-[c:Contains]->(f:Food)
        WHERE f.name = {term}
        WITH DISTINCT r, c
        MATCH (r)-[a:About]->(re:Restaurant)
        RETURN re.name, sum(c.food_sentiment)/count(r) AS average_senti, count(r) AS number
        ORDER BY average_senti DESC
        """

    query_2_pop = """
        WITH {food_list} as foods
        MATCH (r:Review)-[c:Contains]->(f:Food)
        WHERE f.name IN foods
        WITH r, count(distinct c) as co, foods
        WHERE co = size(foods)
        MATCH (r)-[a:About]->(re:Restaurant)
        WITH re, r,foods
        MATCH (r:Review)-[c:Contains]->(f:Food {name:foods[0]})
        RETURN re.name,sum(c.food_sentiment)/count(r) AS average_senti, count(r) AS number
        ORDER BY number DESC
        """

    query_2_rate = """
        WITH {food_list} as foods
        MATCH (r:Review)-[c:Contains]->(f:Food)
        WHERE f.name IN foods
        WITH r, count(distinct c) as co, foods
        WHERE co = size(foods)
        MATCH (r)-[a:About]->(re:Restaurant)
        WITH re, r,foods
        MATCH (r:Review)-[c:Contains]->(f:Food {name:foods[0]})
        RETURN re.name,sum(c.food_sentiment)/count(r) AS average_senti, count(r) AS number
        ORDER BY average_senti DESC
        """

    if sort == 'number_of_reviews':
        query = query_pop
        query_2 = query_2_pop
    if sort == 'rating':
        query = query_rate
        query_2 = query_2_rate

    ans = graph.run(query, term=text).data()

    if ans:
        for item in ans:
            result.append(item['re.name'])
        return result, spli

    else:
        key_words = key_words.split()
        spli = True

        ans = graph.run(query_2, food_list=key_words).data()

        for item in ans:
            result.append(item['re.name'])

        if not result:
            return ['No records found'], spli
        else:
            return result, spli


###### Function here ######
def QueryFunction(food_text, sortBy):
    food_text = food_text.lower()
    res, spli = search_restaurant_by_food(burpple, food_text, sortBy)
    result = query_reviews_food(burpple, res, food_text, spli)
    return result




def Cuisine_query(cuisine_txt, sortBy):
    cuisine_txt = cuisine_txt.lower()
    res = search_restaurant_by_cusine(burpple, cuisine_txt,sortBy)
    result = query_reviews_cuisine(burpple, res)

    return result